In [37]:
# Import necessary libraries 
import pyodbc 
import pandas as pd 
import math 
import matplotlib.pyplot as plt 
import seaborn as sns 
from tqdm.notebook import tqdm  # for visual progress bars in Jupyter 

In [ ]:
# SQL script to create all tables of project 
create_table_scripts = ''' 
    create table accident_record( 
        id int identity primary key, 
        crash_date date, 
        crash_time time 
    ); 
    create table injury_person_type( 
        id int identity primary key, 
        type_name varchar(100)
    ); 
    create table injury_area( 
        id int identity primary key, 
        area_name varchar(50)
    ); 
    create table personal_injury_record( 
        id int identity primary key, 
        injury_result varchar(20), 
        accident_record_id int foreign key references accident_record(id), 
        injury_person_type_id int foreign key references injury_person_type(id), 
        injury_area_id int foreign key references injury_area(id)
    ); 
    create table personal_info( 
        id int primary key foreign key references personal_injury_record(id), 
        age int, 
        sex char(1) 
    ); 
'''

In [47]:
# Create connection string to connect to SQL Server 
server = 'DESKTOP-T0P4LFG\\SQLEXPRESS' 
database = 'task1_part2' 
driver = 'ODBC Driver 17 for SQL Server' 
connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes' 

In [49]:
# Define function to execute SQL statement 
def execute_query(query, connection): 
    cursor = connection.cursor() 
    cursor.execute(query) 
    connection.commit() 
    cursor.close()
    
def execute_query_and_get_one(query, connection):
    cursor = connection.cursor()
    cursor.execute(query)
    record = cursor.fetchone()
    connection.commit()
    cursor.close()

    return record

def execute_insert_and_get_id(query, connection):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.execute('select @@identity as id;')
    id = cursor.fetchone()[0]
    connection.commit()
    cursor.close()

    return id

In [51]:
# Connect to the database and create tables 
with pyodbc.connect(connection_string) as conn: 
    execute_query(create_table_scripts, conn)
    
# Message to the user 
print('All of the tables are created successfully.')

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [63]:
# Import data from file 
data_path = './vehicle_accidents.xlsx' 
data = pd.read_excel(data_path)
data.head(5)

,CRASH_DATE,CRASH_TIME,INJURY_RESULT,PERSON_AGE,INJURY_AREA,PERSON_SEX,INJURY_PERSON_TYPE
0,2022-05-02,21:00:00,Killed,62.0,Head,F,Pedestrian
1,2022-05-21,00:00:00,Killed,24.0,Entire Body,M,Occupant
2,2022-10-15,02:00:00,Killed,30.0,Head,M,Occupant
3,2022-04-17,13:00:00,Killed,71.0,Head,M,Pedestrian
4,2022-05-25,22:00:00,Killed,69.0,Entire Body,F,Pedestrian


In [67]:
# Prepare data for injury_area table 
injury_area_data = data[['INJURY_AREA']].drop_duplicates().reset_index(drop=True) 

# Prepare data for injury_person_type table 
injury_person_type_data = data[['INJURY_PERSON_TYPE']].drop_duplicates().reset_index(drop=True) 

# Insert data into injury_area table 
insert_data_injury_area_scripts = "insert into injury_area(area_name) values " + \
                                  ",".join(f"('{area}')" for area in injury_area_data['INJURY_AREA']) + ";" 
# Insert data into injury_person_type table 
insert_data_injury_person_type_scripts = "insert into injury_person_type(type_name) values " + \
                                         ",".join(f"('{type}')" for type in injury_person_type_data['INJURY_PERSON_TYPE']) + ";" 
# Insert data into tables 
with pyodbc.connect(connection_string) as conn: 
    execute_query(insert_data_injury_area_scripts, conn) 
    execute_query(insert_data_injury_person_type_scripts, conn)

# Message to the user 
print('injury_area and injury_person_type tables are inserted successfully.') 

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [33]:
# Convert 'CRASH_DATE' to datetime if it's not already 
data['CRASH_DATE'] = pd.to_datetime(data['CRASH_DATE']) 

NameError: name 'data' is not defined

In [1]:
# Extract the day of the week and create a new column for it 
data['DAY_OF_WEEK'] = data['CRASH_DATE'].dt.day_name() 

NameError: name 'data' is not defined

In [35]:
# Plot the distribution of accidents by day of the week 
plt.figure(figsize=(10, 6)) 
sns.countplot(x='DAY_OF_WEEK', data=data, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']) 
plt.title('Distribution of Accidents by Day of the Week') 
plt.xlabel('Day of the Week') 
plt.ylabel('Number of Accidents') 
plt.xticks(rotation=45) 
plt.show() 

NameError: name 'data' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
|